# Multiple linear regression model
**Here is how the equation looks**
>ΔR=β0​+β1​(Commercial Airbnb Listings)+β2​(Population Density)+β3​(Distance to Centre)+β4​(Deprivation)+ϵ
>
>Explanation:
>β0\beta_0β0​: Intercept (baseline rental price change when all predictors are zero).
β1,β2,β3,β4\beta_1, \beta_2, \beta_3, \beta_4β1​,β2​,β3​,β4​: Coefficients representing the effect of each predictor on rental price changes.
ϵ\epsilonϵ: Error term that captures variability in ΔR\Delta RΔR not explained by the predictors.



In [3]:
# load packages
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import requests
import zipfile
import matplotlib.cm as cm
import matplotlib.pyplot as plt

In [4]:
# load the clean airbnb data into a pandas dataframe
# get the current working directory
cwd = os.getcwd()
path = f"{cwd}/data/inside_airbnb/inside_airbnb_clean.csv"
# create a dataframe of the raw Inside Airbnb data
df = pd.read_csv(path, low_memory=False)

In [5]:
import pandas as pd

# Reading the data
df_rentalprice = pd.read_csv("https://github.com/a-saveleva/FSDS_Slap/blob/main/private_Rent/onsrents.csv?raw=true")

# Filtering for London boroughs based on Area Code
df_london = df_rentalprice[df_rentalprice['Area code'].str.startswith('E09')]

# Resetting the index of the filtered DataFrame
df_london = df_london.reset_index(drop=True)

# Ensure 'Time period' is in datetime format
df_london['Time period'] = pd.to_datetime(df_london['Time period'], errors='coerce')

# Define the start and end dates of the financial year 2021-22
start_date_2021 = pd.to_datetime("2021-04-01")
end_date_2021 = pd.to_datetime("2022-03-31")

# Filter for the 2021-22 financial year
filtered_rents_2021 = df_london[(df_london['Time period'] >= start_date_2021) & (df_london['Time period'] <= end_date_2021)]

# Grouping by Area code and Area name to calculate the average rental price for 2021-22
monthly_avg_rent_2021 = filtered_rents_2021.groupby(['Area code', 'Area name'])['Rental price'].mean().reset_index()

# Renaming the columns for clarity for 2021-22
monthly_avg_rent_2021.columns = ['Mnemonic', 'Borough Name', 'Average Rental Price 2021-22']

# Define the start and end dates of the financial year 2020-21
start_date_2020 = pd.to_datetime("2020-04-01")
end_date_2020 = pd.to_datetime("2021-03-31")

# Filter for the 2020-21 financial year
filtered_rents_2020 = df_london[(df_london['Time period'] >= start_date_2020) & (df_london['Time period'] <= end_date_2020)]

# Grouping by Area code and Area name to calculate the average rental price for 2020-21
monthly_avg_rent_2020 = filtered_rents_2020.groupby(['Area code', 'Area name'])['Rental price'].mean().reset_index()

# Renaming the columns for clarity for 2020-21
monthly_avg_rent_2020.columns = ['Mnemonic', 'Borough Name', 'Average Rental Price 2020-21']

# Merging the two datasets (2020-21 and 2021-22) by 'Mnemonic' and 'Borough Name'
df_rental_comparison = pd.merge(monthly_avg_rent_2021, monthly_avg_rent_2020, on=['Mnemonic', 'Borough Name'], how='inner')

# Display the result
print(df_rental_comparison)

# Check for missing values in the merged DataFrame
print(df_rental_comparison.isnull().sum())


     Mnemonic            Borough Name  Average Rental Price 2021-22  \
0   E09000002    Barking and Dagenham                   1254.583333   
1   E09000003                  Barnet                   1473.666667   
2   E09000004                  Bexley                   1131.666667   
3   E09000005                   Brent                   1465.166667   
4   E09000006                 Bromley                   1312.750000   
5   E09000007                  Camden                   2076.583333   
6   E09000008                 Croydon                   1214.166667   
7   E09000009                  Ealing                   1572.833333   
8   E09000010                 Enfield                   1344.083333   
9   E09000011               Greenwich                   1453.500000   
10  E09000012                 Hackney                   1911.000000   
11  E09000013  Hammersmith and Fulham                   2078.916667   
12  E09000014                Haringey                   1652.833333   
13  E0

**Reading in the population density data**

In [6]:
df_censuspopdens = pd.read_csv("https://github.com/a-saveleva/FSDS_Slap/blob/main/census_data/Census_PopDensity.csv?raw=true", 
                                  skiprows = 6, nrows = 33 ,usecols = ['mnemonic', '2021'])

df_censuspopdens.rename(columns={'2021':'PopDensity'}, inplace=True)

print(df_censuspopdens)
# Mapping of mnemonic codes to borough names
mnemonic_to_borough = {
    'E09000001': 'City of London',
    'E09000002': 'Barking and Dagenham',
    'E09000003': 'Barnet',
    'E09000004': 'Bexley',
    'E09000005': 'Brent',
    'E09000006': 'Bromley',
    'E09000007': 'Camden',
    'E09000008': 'Croydon',
    'E09000009': 'Ealing',
    'E09000010': 'Enfield',
    'E09000011': 'Greenwich',
    'E09000012': 'Hackney',
    'E09000013': 'Hammersmith and Fulham',
    'E09000014': 'Haringey',
    'E09000015': 'Harrow',
    'E09000016': 'Havering',
    'E09000017': 'Hillingdon',
    'E09000018': 'Hounslow',
    'E09000019': 'Islington',
    'E09000020': 'Kensington and Chelsea',
    'E09000021': 'Kingston upon Thames',
    'E09000022': 'Lambeth',
    'E09000023': 'Lewisham',
    'E09000024': 'Merton',
    'E09000025': 'Newham',
    'E09000026': 'Redbridge',
    'E09000027': 'Richmond upon Thames',
    'E09000028': 'Southwark',
    'E09000029': 'Sutton',
    'E09000030': 'Tower Hamlets',
    'E09000031': 'Waltham Forest',
    'E09000032': 'Wandsworth',
    'E09000033': 'Westminster'
}

# Add a column for borough names
df_censuspopdens['Borough'] = df_censuspopdens['mnemonic'].map(mnemonic_to_borough)

# Display the updated DataFrame
print(df_censuspopdens)


     mnemonic  PopDensity
0   E09000002      6065.8
1   E09000003      4488.7
2   E09000004      4070.7
3   E09000005      7859.6
4   E09000006      2197.7
5   E09000007      9640.9
6   E09000001      2975.0
7   E09000008      4516.0
8   E09000009      6611.6
9   E09000010      4082.5
10  E09000011      6108.6
11  E09000012     13593.3
12  E09000013     11161.1
13  E09000014      8930.2
14  E09000015      5175.4
15  E09000016      2332.3
16  E09000017      2644.0
17  E09000018      5148.0
18  E09000019     14574.9
19  E09000020     11816.5
20  E09000021      4512.1
21  E09000022     11839.1
22  E09000023      8551.5
23  E09000024      5721.6
24  E09000025      9690.1
25  E09000026      5501.9
26  E09000027      3401.9
27  E09000028     10659.0
28  E09000029      4781.0
29  E09000030     15702.9
30  E09000031      7173.3
31  E09000032      9560.0
32  E09000033      9514.2
     mnemonic  PopDensity                 Borough
0   E09000002      6065.8    Barking and Dagenham
1   E09000003   

**Reading in the deprivation data**

In [7]:
df_censusdepriv = pd.read_csv("https://github.com/a-saveleva/FSDS_Slap/blob/main/census_data/Census_Deprivation.csv?raw=true", 
                                  skiprows = 6, nrows = 33, usecols = ['mnemonic', 'Household is not deprived in any dimension',
                                                                       'Household is deprived in one dimension',
                                                                       'Household is deprived in two dimensions',
                                                                       'Household is deprived in three dimensions',
                                                                       'Household is deprived in four dimensions'])
df_censusdepriv.shape
df_censusdepriv.dtypes

p_depriv = [df_censusdepriv['Household is not deprived in any dimension'],df_censusdepriv['Household is deprived in one dimension'],
            df_censusdepriv['Household is deprived in two dimensions'], df_censusdepriv['Household is deprived in three dimensions'],
            df_censusdepriv['Household is deprived in four dimensions']]
df_censusdepriv.dtypes

# Mapping of mnemonic codes to borough names
mnemonic_to_borough = {
    'E09000001': 'City of London',
    'E09000002': 'Barking and Dagenham',
    'E09000003': 'Barnet',
    'E09000004': 'Bexley',
    'E09000005': 'Brent',
    'E09000006': 'Bromley',
    'E09000007': 'Camden',
    'E09000008': 'Croydon',
    'E09000009': 'Ealing',
    'E09000010': 'Enfield',
    'E09000011': 'Greenwich',
    'E09000012': 'Hackney',
    'E09000013': 'Hammersmith and Fulham',
    'E09000014': 'Haringey',
    'E09000015': 'Harrow',
    'E09000016': 'Havering',
    'E09000017': 'Hillingdon',
    'E09000018': 'Hounslow',
    'E09000019': 'Islington',
    'E09000020': 'Kensington and Chelsea',
    'E09000021': 'Kingston upon Thames',
    'E09000022': 'Lambeth',
    'E09000023': 'Lewisham',
    'E09000024': 'Merton',
    'E09000025': 'Newham',
    'E09000026': 'Redbridge',
    'E09000027': 'Richmond upon Thames',
    'E09000028': 'Southwark',
    'E09000029': 'Sutton',
    'E09000030': 'Tower Hamlets',
    'E09000031': 'Waltham Forest',
    'E09000032': 'Wandsworth',
    'E09000033': 'Westminster'
}

# Add a column for borough names
df_censusdepriv['Borough'] = df_censusdepriv['mnemonic'].map(mnemonic_to_borough)

# Display the updated DataFrame
print(df_censusdepriv)


print(df_censusdepriv)

     mnemonic  Household is not deprived in any dimension  \
0   E09000002                                        37.6   
1   E09000003                                        49.6   
2   E09000004                                        48.5   
3   E09000005                                        39.9   
4   E09000006                                        54.6   
5   E09000007                                        47.5   
6   E09000001                                        59.8   
7   E09000008                                        48.0   
8   E09000009                                        46.0   
9   E09000010                                        42.2   
10  E09000011                                        48.2   
11  E09000012                                        45.0   
12  E09000013                                        51.4   
13  E09000014                                        43.3   
14  E09000015                                        48.7   
15  E09000016           

**reading airbnb data and calculating how many listings were vacant for 36 days**

In [8]:
# Reading the data
df_airbnb = pd.read_csv("https://github.com/a-saveleva/FSDS_Slap/blob/main/data/inside_airbnb/inside_airbnb_clean.csv?raw=true")
#print(df_airbnb.head)


# Filter listings with availability over 36 days and hosts who have more than one listing
"""
Data on STL occupancy in London varies. Also, official occupancy cap does not sufficiently help describe "fair use" of Airbnb.
We use the data provided by ONS: The number of guest nights, nights, and stays for short-term lets offered through online collaborative 
economy platforms (Airbnb, Booking.com and Expedia Group). https://www.ons.gov.uk/peoplepopulationandcommunity/housing/datasets/guestnightsnightsandstaysforshorttermletsuk
Matched with Hosts, listings, and bed spaces of short-term lets, UK: 2023. https://www.ons.gov.uk/peoplepopulationandcommunity/housing/datasets/hostslistingsandbedspacesofshorttermletsuk2023
See more in text description of data cleaning and wrangling.

According to this data, in December 2023 (month where there is overlap between 2 datasets), there were 152,050 nights spent in all 111,880 observed STLs in London, 5.84 nights per listing.
We take this as a benchmark for "fair" use of Airbnb in London. Hosts that are available for longer than 5.84*12 days will be considereded as those taking advantage of the opportunity,
which also means there is a possibility that they don't live in the flat.
"""
df_filtered = df_airbnb[(df_airbnb['availability_365'] > (5.84*12))] #& (df_airbnb['host_total_listings_count'] > 1)]

# Display the filtered DataFrame
#print(df_filtered)

boro_shp = os.path.join(cwd, "data", "london-boundaries", "statistical-gis-boundaries-london", "ESRI", "London_Borough_Excluding_MHW.shp")  
boro_gdf = gpd.read_file(boro_shp).to_crs('EPSG:27700')[['NAME', 'GSS_CODE', 'ONS_INNER', 'geometry']]

# Step 1: Convert df_airbnb into a GeoDataFrame with EPSG:4326
df_filtered = gpd.GeoDataFrame(
    df_filtered, 
    geometry=gpd.points_from_xy(df_filtered['longitude'], df_filtered['latitude']),
    crs="EPSG:4326"  # Ensure the CRS is set to WGS84
)

# Step 2: Reproject df_airbnb to match the CRS of boro_gdf (EPSG:27700)
df_filtered = df_filtered.to_crs(boro_gdf.crs)

# Step 3: Verify that both GeoDataFrames have the same CRS
#print(f"Airbnb CRS: {df_filtered.crs}")
#print(f"Borough CRS: {boro_gdf.crs}")

# Step 4: Perform the spatial join
df_filtered = gpd.sjoin(df_filtered, boro_gdf, how="left", predicate='within')

# Step 5: Check the columns in the merged GeoDataFrame
print(df_filtered)

                       id                                      listing_url  \
0                  198258              https://www.airbnb.com/rooms/198258   
1                   33332               https://www.airbnb.com/rooms/33332   
2                   42010               https://www.airbnb.com/rooms/42010   
4                   89870               https://www.airbnb.com/rooms/89870   
6                   96052               https://www.airbnb.com/rooms/96052   
...                   ...                                              ...   
87592  950576237933943040  https://www.airbnb.com/rooms/950576237933943023   
87593  950589815013504256  https://www.airbnb.com/rooms/950589815013504257   
87595  951188392382129024  https://www.airbnb.com/rooms/951188392382129035   
87596  951192793768996992  https://www.airbnb.com/rooms/951192793768996976   
87597  952607914901368448  https://www.airbnb.com/rooms/952607914901368427   

         host_id  host_total_listings_count   latitude  longitu

**merging the datasets**

In [47]:
import pandas as pd
import statsmodels.api as sm

# Merge the population density data
df_merged = pd.merge(df_filtered, df_censuspopdens, left_on='GSS_CODE', right_on='mnemonic', how='left')

# Merge the deprivation data
df_merged = pd.merge(df_merged, df_censusdepriv, left_on='mnemonic', right_on='mnemonic', how='left')

# Ensure the 'Deprivation' score is calculated (sum of the deprivation dimensions)
df_merged['Deprivation'] = (df_merged['Household is deprived in one dimension'] +
                             df_merged['Household is deprived in two dimensions'] +
                             df_merged['Household is deprived in three dimensions'] +
                             df_merged['Household is deprived in four dimensions'])

# Defining the independent variables (X) and the dependent variable (y)
X = df_merged[['host_total_listings_count', 'PopDensity', 'Deprivation']]  # Independent variables
y = df_merged['price']  # Dependent variable

# Adding a constant to the model (for the intercept)
X = sm.add_constant(X)

# Fitting the model using OLS (Ordinary Least Squares)
model = sm.OLS(y, X).fit()

# Display the summary of the regression
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.031
Model:                            OLS   Adj. R-squared:                  0.031
Method:                 Least Squares   F-statistic:                     406.5
Date:                Sat, 30 Nov 2024   Prob (F-statistic):          5.99e-260
Time:                        02:05:51   Log-Likelihood:            -2.7948e+05
No. Observations:               38398   AIC:                         5.590e+05
Df Residuals:                   38394   BIC:                         5.590e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                 

> vif 

In [49]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

#Calculating VIF for each feature in the independent variables
vif_data = pd.DataFrame()
vif_data["Variable"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# Displaying the VIF results
print(vif_data)

                    Variable         VIF
0                      const  144.407279
1  host_total_listings_count    1.000543
2                 PopDensity    1.003870
3                Deprivation    1.003901
